# Try in a class setup

In [ ]:
import os
import numpy as np
import pandas as pd
import panel as pn
import holoviews as hv, param
import panel as pn
from colorcet import cm
from holoviews.operation.datashader import rasterize, shade
import holoviews.operation.datashader as hd
from holoviews import streams
from holoviews.element.tiles import StamenTerrain
import dask
import dask.dataframe as dd
import feather
from holoviews import opts  # noqa
import gc

pn.extension('katex')
hv.extension('bokeh')

# Constants and setup
cmaps = ['bgyw','fire','bgy','bmy','gray','kbc']
disp_cols = ['fare_amount', 'tip_amount', 'trip_distance', 'pickup_datetime', 'dropoff_datetime']
m_dict = {1: 'January', 2: 'February', 3: 'March', 4: 'April',
          5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September',
          10: 'October', 11: 'November', 12: 'December'}
opts = dict(width=450,height=400,xaxis=None,yaxis=None,bgcolor='black',show_grid=False)
dpath = 'data'
fth_fnames = sorted([f for f in os.listdir(dpath) if 'fth' in f])
fnames_2014 = fth_fnames[:12]
fnames_2015 = fth_fnames[12:]

# for widgets and selecting
month = None
trip_dis_bnd = (0,100)
tip_amt_bnd = (0, 200)
fare_amt_bnd = (0, 500)
month_bnd = (1, 12)
hour_l = [i for i in range(24)]

# initial data
df_2014 = pd.read_feather(os.path.join(dpath, fnames_2014[0]))
df_2015 = pd.read_feather(os.path.join(dpath, fnames_2015[0]))

class NYCTaxiExplorer(param.Parameterized):
    month      = param.Integer(1, bounds=(1, 12))
    trip_distance   = param.Range(default=(0, 30), bounds=trip_dis_bnd)    
    tip_amount    = param.Range(default=(0, 200), bounds=tip_amt_bnd)
    fare_amount    = param.Range(default=(0, 500), bounds=fare_amt_bnd)
    pick_drop_location   = param.ObjectSelector(default='dropoff', objects=['dropoff', 'pickup'])
    alpha      = param.Magnitude(default=0.6, doc="Alpha value for the map opacity")
    cmap       = param.ObjectSelector(cm['bgyw'], objects={c:cm[c] for c in cmaps})
    pick_drop_hour   = param.ObjectSelector(default='dropoff', objects=['dropoff', 'pickup'])
    hour       = param.List(hour_l)

    def load_dfs(self):
        global df_2014
        global df_2015
        global month
        month = self.month
        df_2014 = pd.read_feather(os.path.join(dpath, fnames_2014[self.month-1]))
        df_2015 = pd.read_feather(os.path.join(dpath, fnames_2015[self.month-1]))
        
    def points(self):
        global month
        if self.month != month:
            self.load_dfs()
        query = f'trip_distance >= {self.trip_distance[0]} & trip_distance <= {self.trip_distance[1]} &\
          tip_amount >= {self.tip_amount[0]} & tip_amount <= {self.tip_amount[1]} &\
          fare_amount >= {self.fare_amount[0]} & fare_amount <= {self.fare_amount[1]}' + \
        f' & {self.pick_drop_hour}_hour in {self.hour}'
        self.sdf_2014 = df_2014.query(query)
        self.sdf_2015 = df_2015.query(query)
        self.shape_2014 = self.sdf_2014.shape
        self.shape_2015 = self.sdf_2015.shape
        self.points_2014 = hv.Points(self.sdf_2014, kdims=[self.pick_drop_location+'_x', self.pick_drop_location+'_y'])
        self.points_2015 = hv.Points(self.sdf_2015, kdims=[self.pick_drop_location+'_x', self.pick_drop_location+'_y'])
        return self.points_2014, self.points_2015
    
    def title(self):
#         if not hasattr(self, 'df_2014_shape'): self.df_2014_shape = df_2014.shape
        title_2014 = pn.pane.HTML('<h4>{:,} Taxi Rides in {} 2014</h4>'.format(self.shape_2014[0], m_dict[self.month]), width = 500)
        title_2015 = pn.pane.HTML('<h4>{:,} Taxi Rides in {} 2015</h4>'.format(self.shape_2015[0], m_dict[self.month]), width = 500)
        return title_2014, title_2015
    
    def disp_dfs(self):
        pane_2014 = pn.pane.DataFrame(self.sdf_2014.nlargest(10, 'tip_amount')[disp_cols], sizing_mode="stretch_width", width=850)
        pane_2015 = pn.pane.DataFrame(self.sdf_2015.nlargest(10, 'tip_amount')[disp_cols], sizing_mode="stretch_width", width=850)
        return pn.Column("#### 2014 Highest Tipping Rides", pane_2014, "#### 2015 Highest Tipping Rides", pane_2015)
    
    @param.depends('pick_drop_location', 'hour', 'month', 'trip_distance', 'tip_amount', 'pick_drop_hour', watch=True)
    def view(self):
        res = self.points()
        points_2014 = hv.DynamicMap(res[0])
        points_2015 = hv.DynamicMap(res[1])
        tiles_2014 = StamenTerrain().apply.opts(alpha=self.param.alpha, **opts)
        agg_2014 = rasterize(points_2014, x_sampling=1, y_sampling=1, width=600, height=400)
        map_2014 = tiles_2014 * shade(agg_2014, cmap=self.param.cmap)
        tiles_2015 = StamenTerrain().apply.opts(alpha=self.param.alpha, **opts)
        agg_2015 = rasterize(points_2015, x_sampling=1, y_sampling=1, width=600, height=400)
        map_2015 = tiles_2015 * shade(agg_2015, cmap=self.param.cmap)
        titles = self.title()
        d_dfs = self.disp_dfs()
        return pn.Column(pn.Row(pn.Column(titles[0], map_2014), pn.Column(titles[1], map_2015)), d_dfs)
    
taxi = NYCTaxiExplorer(name="NYC Taxi Trips")
# Individual widgets from params
month_s = pn.Param(taxi.param.month, widgets = {
    'month': {'type': pn.widgets.IntSlider, 'value_throttled': True, 'callback_throttle': 500}
})
hour = pn.Param(taxi.param.hour, widgets = {
    'hour': {'type': pn.widgets.MultiSelect, 'value':taxi.param.hour.default, 'options':taxi.param.hour.default, 'size':12, 'value_throttled': True, 'callback_throttle': 500}
})
trip = pn.Param(taxi.param.trip_distance, widgets = {
    'trip_distance': {'type': pn.widgets.RangeSlider, 'value_throttled': trip_dis_bnd, 'callback_throttle': 500}
})
tip = pn.Param(taxi.param.tip_amount, widgets = {
    'tip_amount': {'type': pn.widgets.RangeSlider, 'value_throttled': tip_amt_bnd, 'callback_throttle': 500}
})
fare = pn.Param(taxi.param.fare_amount, widgets = {
    'fare_amount': {'type': pn.widgets.RangeSlider, 'value_throttled': fare_amt_bnd, 'callback_throttle': 500}
})
pick_drop_location = pn.Param(taxi.param.pick_drop_location)
pick_drop_hour = pn.Param(taxi.param.pick_drop_hour)
alpha = map_2014 = pn.Param(taxi.param.alpha)
cmap = pn.Param(taxi.param.cmap)

def make_html_text():
    # tab text
    overv_HTML = pn.pane.HTML("This application's purpose is to visually compare taxi rides recorded by the New York City Taxi & Limousine Commission in 2014 and 2015. Rides can be filtered by month, pickup/dropoff (Pick drop hour) hour of day, trip distance, and tip amounts. The mapped location can be toggled to pickup or dropoff (Pick drop location). Note that data is large and loading graphics can take several seconds.", width = 800)    

    data_HTML = pn.pane.HTML("Data comes from the <a href='https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page'>New York City Taxi & Limousine Commission (TLC)</a>. The taxi trip records include fields capturing pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts. The data used in the attached datasets were collected and provided to the NYC Taxi and Limousine Commission (TLC) by technology providers authorized under the Taxicab & Livery Passenger Enhancement Programs (TPEP/LPEP). The trip data was not created by the TLC, and TLC makes no representations as to the accuracy of these data.", width = 800) 

    motiv_HTML = pn.pane.HTML('I was curious to learn more about the HoloViz ecosystem that Datashader is part of. I decided to use the TLC data for two reasons. First, the data is large and I wanted to gain an understanding of how to work with large datasets. The most performant method I found so far was to filter and subset in pandas rather than use the Holoviews Points.select API. Second, I wondered if there were any trends identifiable due to the rise of ridesharing. Recent data from the TLC no longer includes latitude and longitude so I use data from 2014-2015 which covers the period that Lyft started operating in NYC (late July 2014). Keep in mind that Uber has been operating in NYC since 2011. Legislators and interested parties can use an app like this to better understand how ridesharing is impacting cities. Check out the questions and see if you can can answer them!', width = 800) 

    quest_HTML = pn.pane.HTML("Exploration Questions: <br>Do you notice anything about the number of taxi rides year-over-year? <br>Do you notice any particular dropoff locations that seem associated with higher tip amounts? <br>What destinations are more popular in the evening?", width = 800)

    answ_HTML = pn.pane.HTML("Exploration Questions: <br>In general, taxi rides are in a steady decline. <br>Rides that Jersey City stuck out to me as some higher tipping rides, as does LGA airport. <br>Jersey City again seems to have some evening activity.", width = 800)

    futimp_HTML = pn.pane.HTML("There are currently two outstanding issues with this panel/holoviz app. First, <a href='https://discourse.holoviz.org/t/panel-servable-app-memory-usage/560/8'>there appears to be memory leakage that eventually overwhelms served apps.</a> Second, <a href='https://github.com/holoviz/panel/issues/1259'>the param API in Panel used to make this app is currently unable to throttle sliders</a> which means the app tries to generate a new map as you slide the slider rather than on release. This also severely impacts performance.", width = 800)

    refer_HTML = pn.pane.HTML("<a href='https://holoviz.org/'>HoloViz</a> <br><a href='https://examples.pyviz.org/nyc_taxi/dashboard.html'>Pyviz Starting Example</a>", width = 800)
    
    source_HTML = pn.pane.HTML("The <a href='https://github.com/jmhsi/DATA_608_FinalProject'>Github repo</a> with source code for this app.", width = 800) 
    
    return overv_HTML, data_HTML, motiv_HTML, quest_HTML, answ_HTML, futimp_HTML, refer_HTML, source_HTML

tab_heads = ("Overview", "Data Source", "Motivations", "Suggested Questions", "My Answers", 'Future Improvements', "References", "Source Code",)
tabs = pn.Tabs(*zip(tab_heads, make_html_text()))

# Data widgets and graph widgets
data_widgets = pn.Column(month_s, fare, tip, trip)
graph_widgets = pn.Column(alpha, cmap, pick_drop_location)
hour_widgets = pn.Column(hour, pick_drop_hour)
widgets = pn.Row(data_widgets, hour_widgets, graph_widgets)

# Dataframes
# disp_dfs = '## I place hold' #taxi.disp_dfs

# Map panels
maps = taxi.view

# Whole app
app = pn.Column(tabs, widgets, maps).servable() # , disp_dfs
app